In [ ]:
from urllib.parse import quote, urlparse
import urllib.request
import datetime, os, requests, sys


In [ ]:
def download_file(url, filename, timeout=3600):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
        return True
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))

        try:
            response = requests.Session().get(url, timeout=timeout)
            if(response.status_code!=200):
                print('Error response, code = %d, body = %s' % (response.status_code, response.text))
                return False
        except RequestException as e:
            sys.stdout.write("Couldn't read %s because %s" % (url, e))
            return False

        open(filename + '.tmp', "wb").write(response.content)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(response.content), filename))
        return True

            
class RootArg:
    arg_dict = {
        'v': None,
        't': 0,
        'ps': 0,
        'l': None,
        'bt': None,
        'et': None,
        'startDwell': 0,
        'endDwell': 0,
        'fps': 30
    }
    def __init__(self, v, l, bt, et, **kwargs):
        self.arg_dict['v'] = v
        self.arg_dict['l'] = l
        self.arg_dict['bt'] = bt
        self.arg_dict['et'] = et
        
    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://headless.earthtime.org/#' + "&".join(arg_list)
    
class ThumbnailUrl:
    arg_dict = {
        'root': None,
        'width': None,
        'height': None,
        'format': 'png',
        'fps': 30,
        'tileFormat': 'mp4',
        'startDwell': 0,
        'endDwell': 0,
        'fromScreenshot': True
    }
    
    def __init__(self, root, width, height):
        self.arg_dict['root'] = root
        self.arg_dict['width'] = width
        self.arg_dict['height'] = height
        

    def get(self):
        arg_list = []
        for (key,value) in self.arg_dict.items():
            arg_list.append('%s=%s'%(key,value))
        return 'https://hal40-thumbnailworker.createlab.org/thumbnail?' + "&".join(arg_list)
    

def export_slr_images_from_earthtime(sharelink, odir=None, w='5120', h='2880'):
    years = [x/100.0 for x in range(3,93, 10)]
    dates = [str(x) for x in years]
    celsius = [str(x/2).replace('.','_') for x in range(0,9)]    
    u = urlparse(sharelink)
    v = None
    l = None
    for fragment in u.fragment.split('&'):
        key,val = fragment.split('=')
        if key == 'v':
            v = val
        if key == 'l':
            l = val
    if v is None or l is None:
        print('Malformed sharelink')
        return False

    slr4_present = l.find("slr4")
    if slr4_present < 0:
        print('Missing layer slr4. Will be added')
        l += ',slr4'
        return False
        
    
    if odir is None:
        now = datetime.datetime.now()
        timestamp = int(now.timestamp())
        odir = '%s_%s' % (l.replace(',','_'), timestamp)

    os.makedirs(odir, exist_ok=True)
    thumbnails = []
    i = 0
    for date in dates:
        bt = et = date
        root_arg = RootArg(v,l,bt,et)
        thumbnail_url = ThumbnailUrl(quote(root_arg.get(), safe=''), w, h)
        fname = celsius[i] + '.png'
        thumbnails.append((thumbnail_url.get(), os.path.join(odir,fname)))
        i += 1
    for t in thumbnails:
        download_file(t[0], t[1])
    return True
    
    
    

In [ ]:
#export_slr_images_from_earthtime('https://earthtime.org/explore#v=25.75098,-80.17346,9.056,latLng&t=0.03&ps=50&l=blsat,slr4&bt=0.03&et=0.83')

In [ ]:
export_slr_images_from_earthtime('https://earthtime.org/explore#v=25.75098,-80.17346,9.056,latLng&t=0.03&ps=50&l=blsat,slr4&bt=0.03&et=0.83')